In [2]:
from torch.utils.data import Dataset
import glob
import numpy
import pretty_midi
import os
from tqdm import tqdm
import random
import torch
import torch.nn.functional as F
import numpy as np
import pretty_midi
from einops import rearrange

In [31]:
def tokenize_audio_mono(kick_midi, snare_midi, hihat_midi, kick_dac_l, snare_dac_l, hihat_dac_l):
    midi_vocab_size = 1000+128+4+1 # 1133
    audio_vocal_size = 1024+4+1 # 1029

    midi_tokens = []
    for type, midi in enumerate([kick_midi, snare_midi, hihat_midi]):
        for note in midi.instruments[0].notes:
            onset = int(note.start // 0.005) # Time resolution is 5ms
            vel = int(note.velocity)
            if onset <800: # 4sec
                midi_tokens.append([4+onset, 4+1000+vel, 4+1000+128+type]) # 2 is reserved for start and end tokens.
    midi_tokens.sort(key=lambda x: x[0]) # Sort by onset time.
    midi_tokens = [item for sublist in midi_tokens for item in sublist] # Flatten.
    all_tokens_np = np.ones((1+(kick_dac_l.shape[0]),152+(431+8+1)*3), dtype=np.int32) * 2   # <PAD> token = 2 
    
    all_tokens_np[0,1:len(midi_tokens)+1] = np.array(midi_tokens, dtype=np.int32)
    all_tokens_np[:,0] = 0 # <SOS> token
    
    return midi_tokens, all_tokens_np


In [5]:
kick_midi = pretty_midi.PrettyMIDI('/Users/hwang/DrumSlayer/generated_data/drum_data_test/generated_midi/kick_midi/kick_midi_0.midi')
snare_midi = pretty_midi.PrettyMIDI('/Users/hwang/DrumSlayer/generated_data/drum_data_test/generated_midi/snare_midi/snare_midi_0.midi')
hihat_midi = pretty_midi.PrettyMIDI('/Users/hwang/DrumSlayer/generated_data/drum_data_test/generated_midi/hihat_midi/hihat_midi_0.midi')

audio_dac = np.load('/Users/hwang/DrumSlayer/generated_data/drum_data_test/mixed_loops/0_codes.npy')
audio_dac_l = audio_dac[0, :, :]
audio_dac_l.shape


(9, 345)

In [32]:
midi_token ,all_token = tokenize_audio_mono(kick_midi, snare_midi, hihat_midi, audio_dac_l, audio_dac_l, audio_dac_l)

In [35]:
len(midi_token)

72

In [18]:
midi_token[0:30]

[4,
 1103,
 1134,
 28,
 1106,
 1134,
 41,
 1103,
 1134,
 53,
 1096,
 1132,
 78,
 1107,
 1134,
 128,
 1100,
 1134,
 178,
 1106,
 1133,
 178,
 1108,
 1134,
 203,
 1106,
 1134,
 228,
 1108,
 1134]